In [ ]:
import numpy as np

from mxlpy import Model, Simulator, fit, fns, plot
from mxlpy.carousel import Carousel, ReactionTemplate


def get_sir() -> Model:
    """Create a simple SIR model."""
    return (
        Model()
        .add_variables({"s": 0.9, "i": 0.1, "r": 0.0})
        .add_parameters({"beta": 0.2, "gamma": 0.1})
        .add_reaction(
            "infection",
            fns.mass_action_2s,
            args=["s", "i", "beta"],
            stoichiometry={"s": -1, "i": 1},
        )
        .add_reaction(
            "recovery",
            fns.mass_action_1s,
            args=["i", "gamma"],
            stoichiometry={"i": -1, "r": 1},
        )
    )


carousel = Carousel(
    get_sir(),
    {
        "infection": [
            ReactionTemplate(fn=fns.mass_action_2s, args=["s", "i", "beta"]),
            ReactionTemplate(
                fn=fns.michaelis_menten_2s,
                args=["s", "i", "beta", "km_bs", "km_bi"],
                additional_parameters={"km_bs": 0.1, "km_bi": 1.0},
            ),
        ],
        "recovery": [
            ReactionTemplate(fn=fns.mass_action_1s, args=["i", "gamma"]),
            ReactionTemplate(
                fn=fns.michaelis_menten_1s,
                args=["i", "gamma", "km_gi"],
                additional_parameters={"km_gi": 0.1},
            ),
        ],
    },
)

## Simulate carousel ensemble

In [ ]:
carousel_time_course = carousel.time_course(np.linspace(0, 100, 101))
variables_by_model = carousel_time_course.get_variables_by_model()

fig, ax = plot.one_axes()
plot.line_mean_std(variables_by_model["s"].unstack().T, label="s", ax=ax)
plot.line_mean_std(variables_by_model["i"].unstack().T, label="i", ax=ax)
plot.line_mean_std(variables_by_model["r"].unstack().T, label="r", ax=ax)
ax.legend()
plot.show()

## Fitting

In [ ]:
data = (
    Simulator(get_sir().update_parameters({"beta": 0.3, "gamma": 0.15}))
    .simulate(100, steps=11)
    .get_result()
    .unwrap_or_err()
).variables

data.head()

In [ ]:
res = fit.carousel_time_course(
    carousel,
    p0={
        "beta": 0.1,
        "gamma": 0.1,
        # specific to reaction templates
        # "km_bi": 1.0,
    },
    data=data,
    minimizer=fit.LocalScipyMinimizer(),
)

best = res.get_best_fit().model

fig, ax = plot.one_axes()
plot.lines(
    Simulator(best).simulate(100).get_result().unwrap_or_err().variables,
    ax=ax,
)
plot.reset_prop_cycle(ax=ax)
plot.lines(data, linestyle="dashed", ax=ax, legend=False)
plot.show()

In [ ]:
best_fit = res.get_best_fit()

print(best_fit.best_pars)
print([rxn.fn.__name__ for rxn in best_fit.model.get_raw_reactions().values()])